In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

['/device:CPU:0', '/device:GPU:0']


In [ ]:
!pip install hazm
!pip install transformers

import numpy as np
import pandas as pd
from hazm import *
import collections
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout,MaxPool1D 
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam,RMSprop
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from sklearn.model_selection import train_test_split
from transformers import logging as hf_logging
import logging

tf.get_logger().setLevel(logging.ERROR)
hf_logging.set_verbosity_error()

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Deep/NLP/digimag/fixed_train.csv", sep='	')
test = pd.read_csv("/content/drive/MyDrive/Deep/NLP/digimag/fixed_test.csv", sep='	')
dev = pd.read_csv("/content/drive/MyDrive/Deep/NLP/digimag/fixed_dev.csv", sep='	')

train=train.append(dev)
train.head()

,Unnamed: 0,content,label,label_id
0,0,نمایش تبلیغ لاک اسکرین گوشی های هواوی تعداد زی...,علم و تکنولوژی,3
1,1,شکست Justice League باکس آفیس بازخوردهای طرفدا...,هنر و سینما,5
2,2,کلاسیک بینی؛ شب اتفاق افتاد فیلم شب اتفاق افتا...,هنر و سینما,5
3,3,اپل سراغ رنده چراکه آپگرید سطل آشغال! نسل کامپ...,علم و تکنولوژی,3
4,4,بررسی جزء جزء Ori and the Blind Forest جعبه اب...,بازی ویدیویی,0


In [ ]:
# https://github.com/hooshvare/parsbert

model_name = 'HooshvareLab/bert-fa-base-uncased'

config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)

transformer_model = TFBertModel.from_pretrained(model_name, config = config)

In [ ]:
max_length = 100


y_train = to_categorical(train['label_id'])

x = tokenizer(
    text=train['content'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [ ]:
bert = transformer_model.layers[0]

input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob)
pooled_output = dropout(bert_model, training=False)

output = Dense(units=len(train.label_id.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range))(pooled_output)

model = Model(inputs=inputs, outputs=output)


optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

model.compile(optimizer = optimizer,loss = CategoricalCrossentropy(from_logits = True), metrics = CategoricalAccuracy('accuracy'))

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 100)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       TFBaseModelOutputWithPool 162841344 
_________________________________________________________________
dropout_75 (Dropout)         (None, 768)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 5383      
Total params: 162,846,727
Trainable params: 162,846,727
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# input_ids = sequence tokens !
history = model.fit(
    x={'input_ids': x['input_ids']},
    y=y_train,
    validation_split=0.2,
    batch_size=64,
    epochs=10)

Epoch 1/10
96/96 [==============================] - 160s 1s/step - loss: 0.4730 - accuracy: 0.8656 - val_loss: 0.2412 - val_accuracy: 0.9302
Epoch 2/10
96/96 [==============================] - 138s 1s/step - loss: 0.1868 - accuracy: 0.9507 - val_loss: 0.2270 - val_accuracy: 0.9439
Epoch 3/10
96/96 [==============================] - 138s 1s/step - loss: 0.1259 - accuracy: 0.9684 - val_loss: 0.2175 - val_accuracy: 0.9426
Epoch 4/10
96/96 [==============================] - 138s 1s/step - loss: 0.0875 - accuracy: 0.9780 - val_loss: 0.2070 - val_accuracy: 0.9478
Epoch 5/10
96/96 [==============================] - 138s 1s/step - loss: 0.0637 - accuracy: 0.9840 - val_loss: 0.2135 - val_accuracy: 0.9478
Epoch 6/10
96/96 [==============================] - 139s 1s/step - loss: 0.0448 - accuracy: 0.9892 - val_loss: 0.2198 - val_accuracy: 0.9465
Epoch 7/10
96/96 [==============================] - 139s 1s/step - loss: 0.0320 - accuracy: 0.9923 - val_loss: 0.2352 - val_accuracy: 0.9406
Epoch 8/10
96

In [ ]:
y_test = to_categorical(test['label_id'])

x_test = tokenizer(
    text=test['content'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [ ]:
# Evaluate model
loss_blstm, acc_blstm = model.evaluate({'input_ids': x_test['input_ids']}, y_test)
print('Test Accuracy: %f' % (acc_blstm*100))

27/27 [==============================] - 6s 215ms/step - loss: 0.2944 - accuracy: 0.9413
Test Accuracy: 94.131458


In [ ]:
pred=model.predict({'input_ids': x_test['input_ids']})

In [ ]:
for i in range(10):
  print(test['content'][i])
  print(test['label_id'][i])
  print(np.argmax(pred[i]))

صبحانه رژیمی لاغر صبحانه مهم ترین وعده غذایی طول شناخته انرژی نیاز بدن بیدار خواب تامین می کند می دانستید می توانید مصرف وعده سمتی ببرید مصرف صبحانه رژیمی وزن لاغر شوید؟ واقع توجه مواد موجود وعده حیاتی، صبحانه عامل مهم لاغری تبدیل رژیم غذایی متعادلی مصرف می کنید چربی سوزی دارید، وعده ای طولانی غذا نخوردن می خورید، اهمیت زیادی می کند می توان مواد غذایی مناسبی توجه نکته انتخاب وعده صبحانه ۲۵-۳۵ درصد کالری طول سوزانده مصرف صبحانه متعادل حاوی ۴۰۰-۴۵۰ کالری ۲۰۰۰ کالری طول مصرف بخوانید غذاهایی می توان صبحانه خورد کدامند؟ تحقیقی دانشگاه میسوری دانشمندان شد، می دهد مصرف صبحانه هورمون گرلین مواد شیمیایی مغز اشتها ارتباط افزایش می دهد هورمون باعث احساس سیری فرد کاهش گرسنگی، تمایل خوردن خوراکی ساعات آینده دلیل مصرف صبحانه رژیمی متعادل لاغر رسیدن وزن نظرتان منجر صبحانه رژیمی مقوی سرشار مواد مغذی اندازه کافی مصرف بخوانید رژیم غذایی لاغری کاری نیستصبحانه رژیمی شامل ترکیباتی است؟ صبحانه رژیمی مناسب لاغری شامل ترکیبات است:۱۰ ۱۵ گرم چربی۲۵ گرم فیبر۱۳ ۲۰ گرم پروتئینمتخصصان توصیه ویتامین، مواد معدنی کربو